In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/MasterAchievements', './input/MasterProfiles', './input/Users',
                './input/UserFollowers', './input/UserAchievements']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

# Simple EDA of kaggle Grandmasters
As far as I can tell (and please correct me if I am wrong) the Grandmaster rank was introduced in 2016 along with the promotion of 71 Competition GM's and one Discussions GM (see, for example, the Topic ["Revamping Kaggle Profiles and the User Ranking System"](https://www.kaggle.com/general/20695)). This notebook makes use of the [Meta kaggle dataset](https://www.kaggle.com/datasets/kaggle/meta-kaggle) in conjunction with the ["Meta Kaggle-Master Achievements Snapshot"](https://www.kaggle.com/steubk/meta-kagglemaster-achievements-snapshot) created by [steubk](https://www.kaggle.com/steubk). 

**Notes:** 
* The dataset does not include Grandmasters who have gone on to become kaggle staff.
* Given that this notebook makes use of various datasets it may take a few days for things to filter through: if you have just become a GM (or $n$xGM) then well done: your data will appear here soon!

In [1]:
# import pandas as pd
exec(os.environ['IREWR_IMPORTS'])
# ALEX: remove extra display code
# import datetime
pd.set_option('display.max_rows', None)
# ALEX: remove plotting
# from IPython.display import Markdown

# import matplotlib.pyplot as plt
# import seaborn as sns
# plt.style.use('fivethirtyeight')
# plt.rcParams.update({'font.size': 16})

# read in the data

# old dataset:
#GM_individual_Tiers = pd.read_csv("../input/list-of-kaggle-grandmasters/GM_individual_Tiers.csv")
#GM_individual_Tiers = GM_individual_Tiers.rename(columns = {'Comp_Tier':'Competitions', 
#                                                            'Dset_Tier':'Datasets',
#                                                            'Note_Tier':'Notebooks',
#                                                            'Disc_Tier':'Discussions'})

MasterAchievements = pd.read_csv("./input/MasterAchievements.scaled.csv")
MasterProfiles = pd.read_csv("./input/MasterProfiles.scaled.csv")
# merge the two files together
Master_file = MasterAchievements.merge(MasterProfiles, on = 'UserName')
# select the Grandmasters from the dataset
GM_individual_Tiers = Master_file.query(" Competitions == 'grandmaster' | Datasets == 'grandmaster'| Notebooks == 'grandmaster'| Discussion == 'grandmaster'").copy()
GM_individual_Tiers['GM_tier_count'] = GM_individual_Tiers[GM_individual_Tiers.astype(str) == 'grandmaster'].count(axis=1)

n_GMs  = GM_individual_Tiers.shape[0]
n_4xGM = GM_individual_Tiers.query("GM_tier_count == 4").shape[0]
n_3xGM = GM_individual_Tiers.query("GM_tier_count == 3").shape[0]
n_2xGM = GM_individual_Tiers.query("GM_tier_count == 2").shape[0]

# convert User Names into Display Names
Users        = pd.read_csv("./input/Users.scaled.csv")
# select the GM users
Users_GM     = Users.query("PerformanceTier == 4")
name_mapping = dict(Users_GM[['UserName','DisplayName']].values)

def make_clickable_link(UserName):
    DisplayName = name_mapping.get(UserName)
    return f'<a target="_blank" href="https://www.kaggle.com/{UserName}">{DisplayName}</a>'

# # print a summary
# now = datetime.datetime.now()
# display(Markdown('This notebook is refreshed almost daily; latest run ' + now.strftime('%A %B %d, %Y') ))


# display(Markdown('### Quick summary'))
# display(Markdown('There are  ' + f'{n_GMs:,}' + '  kaggle Grandmasters.'))
# display(Markdown('Out of these there are ' + f'{n_4xGM:,}' + ' quadruple Grandmasters, '
#                                            + f'{n_3xGM:,}' + ' triple Grandmasters, and '
#                                            + f'{n_2xGM:,}' + ' double Grandmasters.'))
# display(Markdown(""))
_ = 'Out of these there are ' + f'{n_4xGM:,}' + ' quadruple Grandmasters, ' \
                                           + f'{n_3xGM:,}' + ' triple Grandmasters, and ' \
                                           + f'{n_2xGM:,}' + ' double Grandmasters.'


# take a look
GM_individual_Tiers["User"] = GM_individual_Tiers.UserName.map(lambda x: make_clickable_link(x))
# ALEX: remove plotting
# GM_individual_Tiers_styled  = GM_individual_Tiers[["User","Competitions","Datasets","Notebooks","Discussion","Country","GM_tier_count"]].sort_values(by='GM_tier_count', ascending=False).style.bar(subset=['GM_tier_count'], vmin=0, color='#ddaa17')
# GM_individual_Tiers_styled.hide_index()
GM_individual_Tiers[["User","Competitions","Datasets","Notebooks","Discussion","Country","GM_tier_count"]].sort_values(by='GM_tier_count', ascending=False)

,User,Competitions,Datasets,Notebooks,Discussion,Country,GM_tier_count
621,"<a target=""_blank"" href=""https://www.kaggle.co...",grandmaster,grandmaster,grandmaster,grandmaster,India,4
1889,"<a target=""_blank"" href=""https://www.kaggle.co...",grandmaster,grandmaster,grandmaster,grandmaster,United States,4
459,"<a target=""_blank"" href=""https://www.kaggle.co...",grandmaster,grandmaster,grandmaster,grandmaster,India,4
75,"<a target=""_blank"" href=""https://www.kaggle.co...",grandmaster,grandmaster,grandmaster,grandmaster,Norway,4
1117,"<a target=""_blank"" href=""https://www.kaggle.co...",grandmaster,grandmaster,grandmaster,grandmaster,United States,4
1096,"<a target=""_blank"" href=""https://www.kaggle.co...",grandmaster,grandmaster,grandmaster,grandmaster,Lithuania,4
1306,"<a target=""_blank"" href=""https://www.kaggle.co...",grandmaster,grandmaster,grandmaster,grandmaster,United States,4
2876,"<a target=""_blank"" href=""https://www.kaggle.co...",expert,grandmaster,grandmaster,grandmaster,Turkey,3
396,"<a target=""_blank"" href=""https://www.kaggle.co...",grandmaster,expert,grandmaster,grandmaster,Brazil,3
1763,"<a target=""_blank"" href=""https://www.kaggle.co...",grandmaster,expert,grandmaster,grandmaster,Germany,3


In [2]:
# ALEX: remove extra display code
# display(Markdown('#### As a table:'))
# display(Markdown(""))

summary = GM_individual_Tiers[['Competitions','Datasets','Notebooks','Discussion']].apply(pd.Series.value_counts)
# ALEX: remove plotting
# summary.loc[['grandmaster','master','expert','contributor']].style.background_gradient(cmap='YlOrBr', axis=None)
summary.loc[['grandmaster','master','expert','contributor']]

,Competitions,Datasets,Notebooks,Discussion
grandmaster,302,44,126,72
master,34,28,42,44
expert,37,80,87,203
contributor,83,304,201,137


and as an alluvial diagram, where we shall take the liberty of colouring the streams using the most numerous Grandmasters collective; the Competitions Grandmasters:

In [3]:
# ALEX: remove plotting
# import plotly.graph_objects as go
# see 
# https://plotly.github.io/plotly.py-docs/generated/plotly.graph_objects.Parcats.html

tier_colours = {'grandmaster':'#ddaa17','master':'#f76629','expert':'#96508e','contributor':'#20beff'}
# create a new column basing the colours on the Competitions GM's.
GM_individual_Tiers["colour_column"] = GM_individual_Tiers["Competitions"]
# replace the tier with the tier colour
with_colours_df = GM_individual_Tiers.replace({"colour_column": tier_colours})
color = with_colours_df.colour_column;

# for a simple plot:
#fig = px.parallel_categories(with_colours_df, 
#                             dimensions=['Competitions','Datasets','Notebooks','Discussions'],
#                             color=with_colours_df["colour_column"])

# Create dimensions
# ALEX: remove plotting
# Competitions_dim = go.parcats.Dimension(
#     values=GM_individual_Tiers.Competitions,
#     categoryorder='array', categoryarray=['grandmaster','master','expert','contributor'], 
#     label="Competitions")

# Datasets_dim = go.parcats.Dimension(
#     values=GM_individual_Tiers.Datasets,
#     categoryorder='array', categoryarray=['grandmaster','master','expert','contributor'], 
#     label="Datasets")

# Notebooks_dim = go.parcats.Dimension(
#     values=GM_individual_Tiers.Notebooks,
#     categoryorder='array', categoryarray=['grandmaster','master','expert','contributor'], 
#     label="Notebooks")

# Discussions_dim = go.parcats.Dimension(
#     values=GM_individual_Tiers.Discussion,
#     categoryorder='array', categoryarray=['grandmaster','master','expert','contributor'], 
#     label="Discussions")

# fig = go.Figure(data = [go.Parcats(dimensions=[Competitions_dim, Datasets_dim, Notebooks_dim, Discussions_dim],
#         line={'color': color,
#               'shape': 'hspline'},
#         #labelfont={'size': 12, 'family': 'Times'},
#         #tickfont={'size': 12, 'family': 'Times'},
#         arrangement='freeform')])

# fig.show();
_ = GM_individual_Tiers.Competitions
_ = GM_individual_Tiers.Datasets
_ = GM_individual_Tiers.Notebooks
_ = GM_individual_Tiers.Discussion

In [4]:
# ALEX: remove extra display code
# display(Markdown('# Country ranking'))
# display(Markdown('Number of Grandmasters by country'))
# display(Markdown(""))

GM_countries = GM_individual_Tiers["Country"].value_counts().to_frame()
GM_countries['Ranking'] = GM_countries.rank(method="min",ascending=False).astype('int')
# ALEX: remove plotting
# GM_countries.sort_values(by='Country', ascending=False).style.bar(subset=['Country'], vmin=0, color='#ddaa17')
GM_countries.sort_values(by='Country', ascending=False)

,count,Ranking
Country,,
Vietnam,6,19
United States,85,1
United Kingdom,17,6
United Arab Emirates,6,19
Ukraine,11,8
Turkey,9,10
Thailand,1,38
Taiwan,2,26
Switzerland,2,26


# GM 'influencers'
A rather arbitrary definition of 'influencer' but here we shall create a ranking of the GM's who have the most GM followers

In [5]:
# Create a list of GM Id
GM_id_list = Users_GM['Id'].tolist()
# read in the Meta Kaggle followers file
followers = pd.read_csv("./input/UserFollowers.scaled.csv")
# filter by GM users. Note: "FollowingUserId" is the Id of the person being followed
GM_users = followers[followers['FollowingUserId'].isin(GM_id_list)]
# now filter by GM followers
GM_user_follower = GM_users[GM_users['UserId'].isin(GM_id_list)]
# remove self-following entries 
GM_user_follower = GM_user_follower[GM_user_follower['UserId'] != GM_user_follower['FollowingUserId']]

Id_mapping = dict(Users_GM[['Id','UserName']].values)
name_mapping = dict(Users_GM[['UserName','DisplayName']].values)

def make_clickable_link_by_Id(Id):
    UserName = Id_mapping.get(Id)
    DisplayName = name_mapping.get(UserName)
    return f'<a target="_blank" href="https://www.kaggle.com/{UserName}">{DisplayName}</a>'

GM_influencers = GM_user_follower["FollowingUserId"].value_counts().to_frame().reset_index()
# rename the columns
GM_influencers.columns = ['GM_Id', 'number_GM_followers']
# create a "Ranking" column
GM_influencers['Ranking'] = GM_influencers.number_GM_followers.rank(method="min",ascending=False).astype('int')
# replace GM_Id by display name
GM_influencers["User"] = GM_influencers.GM_Id.map(lambda x: make_clickable_link_by_Id(x))
# ALEX: remove plotting
# GM_influencers_styled  = GM_influencers[["Ranking","User","number_GM_followers"]].sort_values(by='number_GM_followers', ascending=False).style.bar(subset=['number_GM_followers'], vmin=0, color='#ddaa17')
# GM_influencers_styled.hide_index()
GM_influencers[["Ranking","User","number_GM_followers"]].sort_values(by='number_GM_followers', ascending=False)

,Ranking,User,number_GM_followers
0,1,"<a target=""_blank"" href=""https://www.kaggle.co...",138
1,2,"<a target=""_blank"" href=""https://www.kaggle.co...",103
2,3,"<a target=""_blank"" href=""https://www.kaggle.co...",96
3,4,"<a target=""_blank"" href=""https://www.kaggle.co...",95
4,5,"<a target=""_blank"" href=""https://www.kaggle.co...",94
5,6,"<a target=""_blank"" href=""https://www.kaggle.co...",87
6,7,"<a target=""_blank"" href=""https://www.kaggle.co...",77
7,7,"<a target=""_blank"" href=""https://www.kaggle.co...",77
8,9,"<a target=""_blank"" href=""https://www.kaggle.co...",71
9,10,"<a target=""_blank"" href=""https://www.kaggle.co...",67


and a totally and utterly useless (at the moment) graph made using [NetworkX](https://networkx.org/) 

In [6]:
# ALEX: remove plotting
# import networkx as nx
# fig, ax = plt.subplots(figsize=(20,12))
# G = nx.from_pandas_edgelist(GM_user_follower, 'UserId', 'FollowingUserId', create_using=nx.Graph())
# nx.draw(G, with_labels=False, node_color="#ddaa17")

# Time series
Number of GM over time

(Note: temporarily commented due to possible bug in the `UserAchievements.csv` file (as per https://www.kaggle.com/discussions/product-feedback/438260 )

In [7]:
UserAchievements = pd.read_csv('./input/UserAchievements.scaled.csv')
# Quite a few kaggle staff in this section as Tier 4 (hence PerformanceTier == 5)
GM_Achievements = UserAchievements.query('(Tier == 4) or (Tier == 5)')

# UserAchievements is big, so now delete
del UserAchievements

GM_Achievements["TierAchievementDate"] = pd.to_datetime(GM_Achievements["TierAchievementDate"])
GM_Achievements = GM_Achievements.sort_values(by='TierAchievementDate', ascending=True)
GM_Achievements.set_index('TierAchievementDate', inplace=True)
GM_Achievements["one"] = 1

Users_GM     = Users.query('(PerformanceTier == 4) or (PerformanceTier == 5)')
# new name mapping to now include kaggle staff
name_mapping = dict(Users_GM[['UserName','DisplayName']].values)

Competitions = GM_Achievements.query("AchievementType == 'Competitions'")
Datasets     = GM_Achievements.query("AchievementType == 'Datasets'")
Scripts      = GM_Achievements.query("AchievementType == 'Scripts'")
Discussion   = GM_Achievements.query("AchievementType == 'Discussion'")

UserAchievements_GM_any  = GM_Achievements.drop_duplicates(subset=['UserId'])
# ALEX: make notebook run
# UserAchievements_GM_any  = UserAchievements_GM_any.resample('D').sum().cumsum()
# UserAchievements_GM_comp = Competitions.resample('D').sum().cumsum()
# UserAchievements_GM_data = Datasets.resample('D').sum().cumsum()
# UserAchievements_GM_note = Scripts.resample('D').sum().cumsum()
# UserAchievements_GM_disc = Discussion.resample('D').sum().cumsum()

# ALEX: remove plotting
# fig, ax = plt.subplots(figsize=(20, 7))
# sns.lineplot(data=UserAchievements_GM_comp, x=UserAchievements_GM_comp.index,  y=UserAchievements_GM_comp["one"],  linewidth = 3, color='magenta',label="Competitions")
# sns.lineplot(data=UserAchievements_GM_any, x=UserAchievements_GM_any.index,  y=UserAchievements_GM_any["one"],  linewidth = 3, color='#ddaa17',label="Total GM")
# sns.lineplot(data=UserAchievements_GM_note, x=UserAchievements_GM_note.index,  y=UserAchievements_GM_note["one"],  linewidth = 3, color='green',label="Notebooks")
# sns.lineplot(data=UserAchievements_GM_disc, x=UserAchievements_GM_disc.index,  y=UserAchievements_GM_disc["one"],  linewidth = 3, color='red',label="Discussions")
# sns.lineplot(data=UserAchievements_GM_data, x=UserAchievements_GM_data.index,  y=UserAchievements_GM_data["one"],  linewidth = 3, color='blue',label="Datasets")
# ax.set(xlabel='Year', ylabel='Number of GM')
# # reordering the labels
# handles, labels = plt.gca().get_legend_handles_labels()
# # specify order
# order = [1,0,2,3,4]
# # pass handle & labels lists along with order as below
# plt.legend([handles[i] for i in order], [labels[i] for i in order]);
_ = UserAchievements_GM_any.index
_ = UserAchievements_GM_any["one"]

# Who's got the most Gold?
These tables include kaggle staff
## Competitions "Top 10"

In [8]:
n_top = 10

Competitions_Top_n = Competitions.sort_values(by='TotalGold', ascending=False).head(n_top).copy()
Competitions_Top_n  = pd.merge(Competitions_Top_n, Users_GM, left_on = ['UserId'], right_on = ['Id'], how='left')

# replace GM_Id by display name
Competitions_Top_n["User"] = Competitions_Top_n.UserName.map(lambda x: make_clickable_link(x))
# ALEX: remove plotting
# Competitions_Top_n_styled  = Competitions_Top_n[["User","TotalGold"]].sort_values(by='TotalGold', ascending=False).style.bar(subset=['TotalGold'], vmin=0, color='#ddaa17')
# Competitions_Top_n_styled.hide_index()
Competitions_Top_n[["User","TotalGold"]].sort_values(by='TotalGold', ascending=False)

,User,TotalGold
0,"<a target=""_blank"" href=""https://www.kaggle.co...",63
1,"<a target=""_blank"" href=""https://www.kaggle.co...",41
2,"<a target=""_blank"" href=""https://www.kaggle.co...",33
3,"<a target=""_blank"" href=""https://www.kaggle.co...",28
4,"<a target=""_blank"" href=""https://www.kaggle.co...",25
5,"<a target=""_blank"" href=""https://www.kaggle.co...",25
6,"<a target=""_blank"" href=""https://www.kaggle.co...",24
7,"<a target=""_blank"" href=""https://www.kaggle.co...",23
8,"<a target=""_blank"" href=""https://www.kaggle.co...",23
9,"<a target=""_blank"" href=""https://www.kaggle.co...",22


## Datasets "Top 10"

In [9]:
Datasets_Top_n = Datasets.sort_values(by='TotalGold', ascending=False).head(n_top).copy()
Datasets_Top_n  = pd.merge(Datasets_Top_n, Users_GM, left_on = ['UserId'], right_on = ['Id'], how='left')

# replace GM_Id by display name
Datasets_Top_n["User"] = Datasets_Top_n.UserName.map(lambda x: make_clickable_link(x))
# ALEX: remove plotting
# Datasets_Top_n_styled  = Datasets_Top_n[["User","TotalGold"]].sort_values(by='TotalGold', ascending=False).style.bar(subset=['TotalGold'], vmin=0, color='#ddaa17')
# Datasets_Top_n_styled.hide_index()
Datasets_Top_n[["User","TotalGold"]].sort_values(by='TotalGold', ascending=False)

,User,TotalGold


## Notebooks "Top 10"

In [10]:
Scripts_Top_n = Scripts.sort_values(by='TotalGold', ascending=False).head(n_top).copy()
Scripts_Top_n  = pd.merge(Scripts_Top_n, Users_GM, left_on = ['UserId'], right_on = ['Id'], how='left')

# replace GM_Id by display name
Scripts_Top_n["User"] = Scripts_Top_n.UserName.map(lambda x: make_clickable_link(x))
# ALEX: remove plotting
# Scripts_Top_n_styled  = Scripts_Top_n[["User","TotalGold"]].sort_values(by='TotalGold', ascending=False).style.bar(subset=['TotalGold'], vmin=0, color='#ddaa17')
# Scripts_Top_n_styled.hide_index()
Scripts_Top_n[["User","TotalGold"]].sort_values(by='TotalGold', ascending=False)

,User,TotalGold
0,"<a target=""_blank"" href=""https://www.kaggle.co...",67
1,"<a target=""_blank"" href=""https://www.kaggle.co...",43
2,"<a target=""_blank"" href=""https://www.kaggle.co...",30
3,"<a target=""_blank"" href=""https://www.kaggle.co...",26
4,"<a target=""_blank"" href=""https://www.kaggle.co...",24
5,"<a target=""_blank"" href=""https://www.kaggle.co...",21
6,"<a target=""_blank"" href=""https://www.kaggle.co...",19
7,"<a target=""_blank"" href=""https://www.kaggle.co...",19
8,"<a target=""_blank"" href=""https://www.kaggle.co...",19
9,"<a target=""_blank"" href=""https://www.kaggle.co...",19


## Discussions "Top 10"

In [11]:
Discussion_Top_n = Discussion.sort_values(by='TotalGold', ascending=False).head(n_top).copy()
Discussion_Top_n  = pd.merge(Discussion_Top_n, Users_GM, left_on = ['UserId'], right_on = ['Id'], how='left')

# replace GM_Id by display name
Discussion_Top_n["User"] = Discussion_Top_n.UserName.map(lambda x: make_clickable_link(x))
# ALEX: remove plotting
# Discussion_Top_n_styled  = Discussion_Top_n[["User","TotalGold"]].sort_values(by='TotalGold', ascending=False).style.bar(subset=['TotalGold'], vmin=0, color='#ddaa17')
# Discussion_Top_n_styled.hide_index()
Discussion_Top_n[["User","TotalGold"]].sort_values(by='TotalGold', ascending=False)

,User,TotalGold
0,"<a target=""_blank"" href=""https://www.kaggle.co...",496
1,"<a target=""_blank"" href=""https://www.kaggle.co...",457
2,"<a target=""_blank"" href=""https://www.kaggle.co...",249
3,"<a target=""_blank"" href=""https://www.kaggle.co...",154
4,"<a target=""_blank"" href=""https://www.kaggle.co...",151
5,"<a target=""_blank"" href=""https://www.kaggle.co...",151
6,"<a target=""_blank"" href=""https://www.kaggle.co...",140
7,"<a target=""_blank"" href=""https://www.kaggle.co...",121
8,"<a target=""_blank"" href=""https://www.kaggle.co...",117
9,"<a target=""_blank"" href=""https://www.kaggle.co...",83


# Related kaggle notebooks
* ["Kaggle Grand/Masters Map"](https://www.kaggle.com/steubk/kaggle-grand-masters-map) written by [steubk](https://www.kaggle.com/steubk)
* ["Kaggle Grand/Masters Map (Notebooks)"](https://www.kaggle.com/code/steubk/kaggle-grand-masters-map-notebooks) written by [steubk](https://www.kaggle.com/steubk)
* ["Kaggle Grand/Masters Map (Competitions)"](https://www.kaggle.com/code/steubk/kaggle-grand-masters-map-competitions) written by [steubk](https://www.kaggle.com/steubk)
* ["Kaggle Grand/Masters Map (Discussion)"](https://www.kaggle.com/code/steubk/kaggle-grand-masters-map-discussion) written by [steubk](https://www.kaggle.com/steubk)
* ["Kaggle Grand/Masters Map (Datasets)"](https://www.kaggle.com/code/steubk/kaggle-grand-masters-map-datasets) written by [steubk](https://www.kaggle.com/steubk)
* ["Kaggle Grandmasters Map"](https://www.kaggle.com/code/gpreda/kaggle-grandmasters-map) written by [Gabriel Preda](https://www.kaggle.com/gpreda)
* ["Kaggle in Numbers"](https://www.kaggle.com/carlmcbrideellis/kaggle-in-numbers)
* ["Active veteran kaggle users"](https://www.kaggle.com/carlmcbrideellis/active-veteran-kaggle-users)
* ["A Meta Kaggler's Guide To Kaggle"](https://www.kaggle.com/steubk/a-meta-kaggler-s-guide-to-kaggle) written by [steubk](https://www.kaggle.com/steubk) 
* ["Meta Kaggle-Master Achievements Snapshot"](https://www.kaggle.com/steubk/meta-kagglemaster-achievements-snapshot) dataset by [steubk](https://www.kaggle.com/steubk) 
* ["Become GrandMaster"](https://www.kaggle.com/raenish/become-grandmaster) written by Raenish David
* ["EDA: Who will be the Next Kaggle Grandmaster?"](https://www.kaggle.com/japandata509/eda-who-will-be-the-next-kaggle-grandmaster) written by [Kaito](https://www.kaggle.com/japandata509)
* ["Meet The Grandmasters"](https://www.kaggle.com/sahidvelji/meet-the-grandmasters) written by [Sahid Velji](https://www.kaggle.com/sahidvelji)
* ["Meta Kaggle: Count User Activities"](https://www.kaggle.com/code/jtrotman/meta-kaggle-count-user-activities) written by [James Trotman](https://www.kaggle.com/jtrotman)
* ["Meta Kaggle Prize Money"](https://www.kaggle.com/datasets/jpmiller/meta-kaggle-moneyboard) - Details on competition prizes and the Kagglers who won them, a dataset by [JohnM](https://www.kaggle.com/jpmiller)